In [45]:
from langchain_core.messages import HumanMessage

from dotenv import load_dotenv
load_dotenv()
import os 

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")
os.environ['LANGSMITH_API_KEY'] = os.getenv("LANGSMITH_API_KEY")
os.environ['LANGSMITH_PROJECT'] = os.getenv("LANGSMITH_PROJECT")
os.environ['LANGSMITH_TRACING'] = os.getenv("LANGSMITH_TRACING")
os.environ['GOOGLE_API_KEY'] = os.getenv("GOOGLE_API_KEY")

In [46]:
from langchain_groq import ChatGroq 
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI

# llm = ChatGroq(model="deepseek-r1-distill-llama-70b")

# llm = ChatOpenAI(model="gpt-4o-mini")
# llm = ChatGoogleGenerativeAI(
#     model="gemini-2.0-flash",
#     temperature=0,
#     max_tokens=None,
#     timeout=None,
#     max_retries=2
# )

# llm = ChatOpenAI(model = "gpt-4o-mini")

llm = ChatGroq(model="qwen-2.5-32b")

# llm = ChatGroq(model="llama3-70b-8192", 
#                timeout=None, 
#                max_retries=2)

In [47]:
from langchain_core.prompts import PromptTemplate 
from langchain_core.output_parsers import JsonOutputParser 

prompt = PromptTemplate(
    template="{system_prompt} \n \n {human_query} \n",
    input_variables= ["system_prompt", "human_query",],
)

In [48]:
user_stories =[{'story_id': 'US-001',
  'title': 'Secure Account Access with Multi-Factor Authentication',
  'description': 'As a user, I want to secure my account with multi-factor authentication, so that I can protect my financial information from unauthorized access.',
  'acceptance_criteria': ['User can enable biometric authentication (fingerprint and facial recognition).',
   'User can set up MPIN for additional security.',
   'System requires multi-factor authentication for each login.',
   'User can reset MPIN through a secure process if forgotten.']},
 {'story_id': 'US-002',
  'title': 'Link and Manage Multiple Bank Accounts',
  'description': 'As a user, I want to link multiple bank accounts to my PayMate profile, so that I can easily manage and transfer funds from any linked account.',
  'acceptance_criteria': ['User can add multiple bank accounts to their profile.',
   'User can select a default bank account for transactions.',
   'System displays all linked bank accounts with balance information.',
   'User can remove or edit linked bank accounts securely.']},
 {'story_id': 'US-003',
  'title': 'Access Instant Micro-Loans',
  'description': 'As a user, I want to access instant micro-loans through the app, so that I can cover immediate financial needs with minimal documentation.',
  'acceptance_criteria': ['User can check eligibility for micro-loans within the app.',
   'User can submit required minimal documentation for loan approval.',
   'System displays loan details including amount, tenure, and interest.',
   'User receives loan amount in their bank account upon approval.']},
 {'story_id': 'US-004',
  'title': 'Pay Utility Bills via PayMate',
  'description': 'As a user, I want to pay utility bills directly through the app, so that I can manage all my payments in one place.',
  'acceptance_criteria': ['User can select from various utility providers (electricity, water, gas, broadband).',
   'User can enter the bill account number to fetch the bill amount.',
   'User can make instant payments using any linked bank account.',
   'System provides a payment confirmation and receipt post-transaction.']},
 {'story_id': 'US-005',
  'title': 'Purchase Insurance Through the App',
  'description': 'As a user, I want to purchase insurance through the app, so that I can protect myself and my family from unforeseen events.',
  'acceptance_criteria': ['User can browse and select different insurance products (e.g., life, health, auto).',
   'User can view insurance details including coverage, premium, and terms.',
   'User can purchase insurance directly through the app with required documentation.',
   'System provides confirmation and policy documents post-purchase.']}]


## Functional documents

In [49]:
functional_document_system_prompt = """
YOU ARE A SENIOR BUSINESS ANALYST AND FUNCTIONAL DESIGN EXPERT WITH EXTENSIVE EXPERIENCE IN THE SOFTWARE DEVELOPMENT LIFE CYCLE (SDLC) AND A STRONG TRACK RECORD OF DELIVERING WORLD-CLASS FUNCTIONAL SPECIFICATION DOCUMENTS (FSD) FOR ENTERPRISE SOFTWARE PROJECTS. RETURN THE OUTPUT IN MARKDOWN FORMAT ONLY. 

YOUR TASK IS TO TRANSLATE PROVIDED USER STORIES INTO A **COMPREHENSIVE, PROFESSIONAL FUNCTIONAL SPECIFICATION DOCUMENT** FOR THE DESIGN PHASE. 

###INSTRUCTIONS###

- CONVERT GIVEN USER STORIES INTO A FORMAL FUNCTIONAL SPECIFICATION DOCUMENT (FSD)
- INCLUDE THE FOLLOWING SECTIONS:
   - **1. INTRODUCTION:** PURPOSE, PROJECT SCOPE, AND SYSTEM OVERVIEW
   - **2. BUSINESS CONTEXT:** PROJECT BACKGROUND, BUSINESS NEEDS, AND OBJECTIVES
   - **3. STAKEHOLDER ANALYSIS:** IDENTIFY PRIMARY STAKEHOLDERS AND USERS IMPACTED
   - **4. FUNCTIONAL REQUIREMENTS:** DETAILED REQUIREMENTS WITH UNIQUE IDENTIFIERS (FR-1, FR-2, etc.)
   - **5. USE CASES / WORKFLOWS:** UML DIAGRAMS OR TEXTUAL FLOWS (ACTIVITY/SEQUENCE DIAGRAMS PREFERRED)
   - **6. DATA REQUIREMENTS:** INPUT FIELDS, OUTPUT FIELDS, VALIDATION RULES, AND DATA FORMATS
   - **7. NON-FUNCTIONAL REQUIREMENTS (NFRs):** PERFORMANCE, SECURITY, SCALABILITY, USABILITY, LEGAL, ETC.
   - **8. DEPENDENCIES & ASSUMPTIONS:** INTERNAL, EXTERNAL, TECHNICAL, OR BUSINESS DEPENDENCIES
   - **9. EDGE CASES & EXCEPTION HANDLING:** POTENTIAL FAILURE POINTS, ALTERNATE FLOWS, AND LIMITATIONS
   - **10. ACCEPTANCE CRITERIA:** AGGREGATE ACCEPTANCE CRITERIA FROM USER STORIES IN A CHECKLIST FORMAT
   - **11. GLOSSARY & DEFINITIONS:** DEFINE ALL BUSINESS TERMS, ROLES, ACRONYMS, AND DOMAIN-SPECIFIC TERMINOLOGY
   - **OPTIONAL: TRACEABILITY MATRIX:** MAP USER STORIES TO THEIR CORRESPONDING FUNCTIONAL REQUIREMENTS

- ALIGN DOCUMENT TO BE USEFUL FOR BOTH **BUSINESS STAKEHOLDERS** AND **TECHNICAL TEAMS** INVOLVED IN THE DESIGN PHASE
- MAINTAIN A FORMAL, EXECUTIVE-READY TONE WITH CLEAR AND CONCISE LANGUAGE
- FOLLOW THE "CHAIN OF THOUGHTS" PROCESS METICULOUSLY BEFORE PRODUCING THE FINAL DOCUMENT
- WHERE APPROPRIATE, OFFER PROFESSIONAL RECOMMENDATIONS IF GAPS, RISKS, OR MISSING ASSUMPTIONS ARE IDENTIFIED
- THE FINAL OUTPUT SHOULD BE BETWEEN **1200 TO 1500 WORDS**.

###CHAIN OF THOUGHTS###

1. UNDERSTAND:
   1.1. DEEPLY ANALYZE THE USER STORIES TO UNCOVER SYSTEM OBJECTIVES, USER INTENTIONS, AND BUSINESS VALUE.
   1.2. DETERMINE THE PRIMARY AUDIENCE, THEIR PAIN POINTS, AND THE IMPACT THIS SYSTEM WILL HAVE ON THEIR WORKFLOW.

2. FRAME:
   2.1. DESIGN THE STRUCTURE OF THE DOCUMENT FOLLOWING INDUSTRY STANDARDS FOR FUNCTIONAL SPECIFICATIONS.
   2.2. LIST ALL STAKEHOLDER GROUPS WHO WILL RELY ON THIS DOCUMENT.

3. EXTRACT:
   3.1. TRANSLATE EACH USER STORY INTO PRECISE FUNCTIONAL REQUIREMENTS (FR) WITH TRACEABLE IDs.
   3.2. FOR EACH FR, INCLUDE THE TRIGGER, SYSTEM RESPONSE, USER ACTIONS, AND ASSOCIATED DATA ELEMENTS.

4. REFINE:
   4.1. VALIDATE THAT EACH REQUIREMENT CONTRIBUTES TO MEETING THE PROJECT OBJECTIVES.
   4.2. IDENTIFY ANY NON-FUNCTIONAL CONSTRAINTS THAT MAY AFFECT USER EXPERIENCE OR SYSTEM PERFORMANCE.

5. DETAIL:
   5.1. PROVIDE COMPLETE, WELL-ORGANIZED SECTIONS, INCLUDING DIAGRAMS, TABLES, OR LISTS WHERE USEFUL.
   5.2. ENSURE THAT EDGE CASES AND EXCEPTION HANDLING ARE FULLY COVERED.

6. ENHANCE:
   6.1. INCLUDE A GLOSSARY TO FACILITATE UNDERSTANDING ACROSS TECHNICAL AND NON-TECHNICAL STAKEHOLDERS.
   6.2. OFFER OPTIONAL TRACEABILITY MATRIX FOR GREATER ACCOUNTABILITY AND TRACKING.

7. FINAL ANSWER:
   7.1. OUTPUT A HIGH-QUALITY, BUSINESS-READY FUNCTIONAL SPECIFICATION DOCUMENT THAT IS FULLY TRACEABLE TO THE INPUT USER STORIES AND READY FOR HANDOVER TO DESIGN/DEVELOPMENT TEAMS.

###WHAT NOT TO DO###

DO NOT:
- CREATE GENERIC OR SUPERFICIAL REQUIREMENTS WITHOUT CONNECTION TO USER STORIES.
- OMIT ACCEPTANCE CRITERIA, EDGE CASES, OR NON-FUNCTIONAL REQUIREMENTS.
- MIX TECHNICAL (CODE-LEVEL) DETAILS OR ARCHITECTURE INTO THE DOCUMENT.
- USE INFORMAL OR OVERLY TECHNICAL JARGON THAT BUSINESS STAKEHOLDERS CANNOT EASILY UNDERSTAND.
- IGNORE TRACEABILITY BETWEEN USER STORIES, REQUIREMENTS, AND BUSINESS GOALS.

###FEW-SHOT EXAMPLES###

####USER STORY INPUT:
AS A **registered user**, I WANT **to reset my password via an email verification process**, SO THAT **I can regain access to my account securely if I forget my password**.

####DESIRED FUNCTIONAL SPECIFICATION OUTPUT:

**1. INTRODUCTION**  
This document defines the functional requirements for the Password Reset Feature of the User Management System.

**2. BUSINESS CONTEXT**  
The business needs a secure mechanism for users to recover access to their accounts without compromising security, improving customer satisfaction and retention.

**3. FUNCTIONAL REQUIREMENTS**  
FR-1: The system shall provide a "Forgot Password" option on the login page.  
FR-2: The system shall send a time-limited reset link to the user's registered email address.  
FR-3: The system shall validate new passwords against password policy rules.  
FR-4: The system shall notify users of successful password reset and redirect them to the login page.

**4. USE CASES/WORKFLOWS**  
[Insert basic UML Use Case Diagram or Textual Workflow]

**5. DATA REQUIREMENTS**  
- Email address input (validated for format)  
- Password input (min 8 characters, 1 special character, 1 number)  
- Token expiration timestamp

**6. NON-FUNCTIONAL REQUIREMENTS**  
- The reset link shall expire within 24 hours.  
- The system must handle over 1000 concurrent password reset requests per minute.

**7. DEPENDENCIES & ASSUMPTIONS**  
- Email server availability  
- Users must have a valid registered email

**8. EDGE CASES & EXCEPTION HANDLING**  
- If a reset token is expired, show an "Expired link" message and provide an option to request a new reset link.  
- If an unregistered email is provided, show a generic "If your email is valid, a reset link will be sent" message.

**9. ACCEPTANCE CRITERIA**  
[Port acceptance criteria directly from the user story]

**10. GLOSSARY & DEFINITIONS**  
- Token: A temporary, secure string used to validate password reset requests.
"""

In [50]:
## documents
from pydantic import BaseModel, Field
class DocumentSection(BaseModel):
    title: str = Field(description="The title of the section")
    content: str = Field(description="The content of the section")
    

In [51]:
class DocumentSection(BaseModel):
    title: str = Field(description="The title of the section")
    content: str = Field(description="The content of the section")

In [52]:
def generate_functional_documents_from_llm(user_stories):
    user_query =  f"Create a functional documents for the user stories: {user_stories}."
    chain = prompt | llm 
    response = chain.invoke({"system_prompt" : functional_document_system_prompt, "human_query" : user_query})
    return response

In [53]:
functional_documents = generate_functional_documents_from_llm(user_stories)

In [58]:
functional_documents

AIMessage(content='# Functional Specification Document (FSD)\n\n## 1. Introduction\n\n### Purpose\nThis document outlines the functional requirements for the PayMate application, focusing on enhancing user security, account management, financial services, and utility payments. The purpose is to define the functionalities that will be implemented to meet the business needs and user stories.\n\n### Project Scope\nThe project scope includes developing and integrating features that allow users to secure their accounts, manage multiple bank accounts, access instant micro-loans, pay utility bills, and purchase insurance directly through the PayMate app.\n\n### System Overview\nPayMate is a mobile application designed to provide users with seamless financial management, including secure account access, multi-account management, financial services, and utility payments.\n\n## 2. Business Context\n\n### Project Background\nPayMate aims to provide a comprehensive financial management tool that m

In [59]:
functional_documents.content

'# Functional Specification Document (FSD)\n\n## 1. Introduction\n\n### Purpose\nThis document outlines the functional requirements for the PayMate application, focusing on enhancing user security, account management, financial services, and utility payments. The purpose is to define the functionalities that will be implemented to meet the business needs and user stories.\n\n### Project Scope\nThe project scope includes developing and integrating features that allow users to secure their accounts, manage multiple bank accounts, access instant micro-loans, pay utility bills, and purchase insurance directly through the PayMate app.\n\n### System Overview\nPayMate is a mobile application designed to provide users with seamless financial management, including secure account access, multi-account management, financial services, and utility payments.\n\n## 2. Business Context\n\n### Project Background\nPayMate aims to provide a comprehensive financial management tool that meets the diverse n

### revised the functional document

In [74]:
revised_functional_document_system_prompt = """
YOU ARE A SENIOR BUSINESS ANALYST AND FUNCTIONAL DESIGN EXPERT WITH EXTENSIVE EXPERIENCE IN THE SOFTWARE DEVELOPMENT LIFE CYCLE (SDLC) AND A STRONG TRACK RECORD OF DELIVERING WORLD-CLASS FUNCTIONAL SPECIFICATION DOCUMENTS (FSD) FOR ENTERPRISE SOFTWARE PROJECTS. RETURN THE OUTPUT IN MARKDOWN FORMAT ONLY.

YOUR TASK IS TO REVISE THE FUNCTIONAL DOCUMENTATION BASED ON THE PROVIDED USER FEEDBACK AND THE EXISTING FUNCTIONAL DOCUMENT.

### INSTRUCTIONS ###

- REVISE THE FUNCTIONAL DOCUMENT TO INCORPORATE THE USER FEEDBACK IN A **CLEAR, CONCISE, AND PROFESSIONAL** FORMAT.
- THE REVISED DOCUMENT SHOULD INCLUDE THE FOLLOWING SECTIONS:
  - **1. INTRODUCTION:** PURPOSE, PROJECT SCOPE, AND SYSTEM OVERVIEW
  - **2. BUSINESS CONTEXT:** PROJECT BACKGROUND, BUSINESS NEEDS, AND OBJECTIVES
  - **3. STAKEHOLDER ANALYSIS:** IDENTIFY PRIMARY STAKEHOLDERS AND USERS IMPACTED
  - **4. FUNCTIONAL REQUIREMENTS:** DETAILED REQUIREMENTS WITH UNIQUE IDENTIFIERS (FR-1, FR-2, etc.)
  - **5. USE CASES / WORKFLOWS:** UML DIAGRAMS OR TEXTUAL FLOWS (ACTIVITY/SEQUENCE DIAGRAMS PREFERRED)
  - **6. DATA REQUIREMENTS:** INPUT FIELDS, OUTPUT FIELDS, VALIDATION RULES, AND DATA FORMATS
  - **7. NON-FUNCTIONAL REQUIREMENTS (NFRs):** PERFORMANCE, SECURITY, SCALABILITY, USABILITY, LEGAL, ETC.
  - **8. DEPENDENCIES & ASSUMPTIONS:** INTERNAL, EXTERNAL, TECHNICAL, OR BUSINESS DEPENDENCIES
  - **9. EDGE CASES & EXCEPTION HANDLING:** POTENTIAL FAILURE POINTS, ALTERNATE FLOWS, AND LIMITATIONS
  - **10. ACCEPTANCE CRITERIA:** AGGREGATE ACCEPTANCE CRITERIA FROM USER STORIES IN A CHECKLIST FORMAT
  - **11. GLOSSARY & DEFINITIONS:** DEFINE ALL BUSINESS TERMS, ROLES, ACRONYMS, AND DOMAIN-SPECIFIC TERMINOLOGY
  - **OPTIONAL: TRACEABILITY MATRIX:** MAP USER STORIES TO THEIR CORRESPONDING FUNCTIONAL REQUIREMENTS

- ALIGN THE DOCUMENT TO BE USEFUL FOR BOTH **BUSINESS STAKEHOLDERS** AND **TECHNICAL TEAMS** INVOLVED IN THE DESIGN PHASE
- MAINTAIN A FORMAL, EXECUTIVE-READY TONE WITH CLEAR AND CONCISE LANGUAGE
- FOLLOW THE "CHAIN OF THOUGHTS" PROCESS METICULOUSLY BEFORE PRODUCING THE FINAL DOCUMENT
- WHERE APPROPRIATE, OFFER PROFESSIONAL RECOMMENDATIONS IF GAPS, RISKS, OR MISSING ASSUMPTIONS ARE IDENTIFIED
- THE FINAL OUTPUT SHOULD BE BETWEEN **1200 TO 1500 WORDS**.

### CHAIN OF THOUGHTS ###

1. UNDERSTAND:
   1.1. REVIEW AND COMPREHEND THE EXISTING FUNCTIONAL DOCUMENT AND THE USER FEEDBACK PROVIDED.
   1.2. IDENTIFY CHANGES OR IMPROVEMENTS NEEDED BASED ON THE USER'S COMMENTS AND REQUIREMENTS.

2. FRAME:
   2.1. ORGANIZE THE DOCUMENT IN A LOGICAL, INDUSTRY STANDARD STRUCTURE.
   2.2. IDENTIFY ALL STAKEHOLDER GROUPS WHO WILL RELY ON THIS DOCUMENT AND ACCOUNT FOR THEIR NEEDS IN THE REVISED VERSION.

3. EXTRACT:
   3.1. IDENTIFY THE IMPACT OF THE FEEDBACK ON EACH FUNCTIONAL REQUIREMENT AND ADJUST AS NECESSARY.
   3.2. ENSURE ALL FUNCTIONAL REQUIREMENTS ARE CLEARLY DEFINED AND TRACEABLE TO THE USER STORIES.

4. REFINE:
   4.1. VALIDATE THAT THE REVISED REQUIREMENTS ARE ALIGNED WITH THE PROJECT OBJECTIVES AND USER NEEDS.
   4.2. REVIEW THE NON-FUNCTIONAL REQUIREMENTS TO ENSURE THAT THEY MEET PERFORMANCE AND USABILITY EXPECTATIONS.

5. DETAIL:
   5.1. UPDATE DIAGRAMS, TABLES, OR LISTS AS NECESSARY TO CLARIFY THE UPDATED REQUIREMENTS.
   5.2. ENSURE EDGE CASES AND EXCEPTION HANDLING ARE ADEQUATELY COVERED BASED ON THE FEEDBACK.

6. ENHANCE:
   6.1. INCLUDE A GLOSSARY TO HELP STAKEHOLDERS, BOTH TECHNICAL AND NON-TECHNICAL, UNDERSTAND THE TERMINOLOGY.
   6.2. CONSIDER INCLUDING A TRACEABILITY MATRIX FOR GREATER ACCOUNTABILITY AND TRACKING OF REQUIREMENTS.

7. FINAL ANSWER:
   7.1. OUTPUT A REVISED, HIGH-QUALITY FUNCTIONAL SPECIFICATION DOCUMENT THAT FULLY ADDRESSES USER FEEDBACK AND IS READY FOR HANDOVER TO DESIGN/DEVELOPMENT TEAMS.

### WHAT NOT TO DO ###

DO NOT:
- LEAVE REQUIREMENTS AMBIGUOUS OR NOT FULLY ADDRESSED BASED ON USER FEEDBACK.
- OMIT ACCEPTANCE CRITERIA, EDGE CASES, OR NON-FUNCTIONAL REQUIREMENTS IN THE REVISED DOCUMENT.
- INCLUDE CODE-LEVEL DETAILS OR ARCHITECTURE IN THE DOCUMENT UNLESS SPECIFICALLY REQUESTED.
- USE INFORMAL LANGUAGE OR OVERLY TECHNICAL JARGON THAT MAY BE DIFFICULT FOR BUSINESS STAKEHOLDERS TO UNDERSTAND.
- IGNORE TRACEABILITY BETWEEN USER STORIES, REQUIREMENTS, AND BUSINESS GOALS.
"""

In [75]:
user_feedback = "The user wants to reset their password via an email verification process."

def revised_functional_document_from_llm(functional_document, user_feedback):
    user_query =  f"Revise the functional document : {functional_document} and by following the user feedback: {user_feedback}. and return the complete revised functional document. "
    chain = prompt | llm 
    response = chain.invoke({"system_prompt" : revised_functional_document_system_prompt, "human_query" : user_query})
    return response

In [76]:
revised_functional_document = revised_functional_document_from_llm(functional_documents.content, user_feedback)
revised_functional_document

AIMessage(content='# Functional Specification Document (FSD)\n\n## 1. Introduction\n\n### Purpose\nThis document outlines the functional requirements for the PayMate application, focusing on enhancing user security, account management, financial services, utility payments, and user account recovery. The purpose is to define the functionalities that will be implemented to meet the business needs and user stories effectively.\n\n### Project Scope\nThe project scope includes developing and integrating features that allow users to secure their accounts, manage multiple bank accounts, access instant micro-loans, pay utility bills from various providers, purchase insurance directly through the app, and reset their passwords via email verification.\n\n### System Overview\nPayMate is a mobile application designed to provide users with seamless financial management, including secure account access, multi-account management, financial services, utility payments, and a secure method for resetting

In [77]:
revised_functional_document.content

'# Functional Specification Document (FSD)\n\n## 1. Introduction\n\n### Purpose\nThis document outlines the functional requirements for the PayMate application, focusing on enhancing user security, account management, financial services, utility payments, and user account recovery. The purpose is to define the functionalities that will be implemented to meet the business needs and user stories effectively.\n\n### Project Scope\nThe project scope includes developing and integrating features that allow users to secure their accounts, manage multiple bank accounts, access instant micro-loans, pay utility bills from various providers, purchase insurance directly through the app, and reset their passwords via email verification.\n\n### System Overview\nPayMate is a mobile application designed to provide users with seamless financial management, including secure account access, multi-account management, financial services, utility payments, and a secure method for resetting passwords.\n\n## 

## technical document

In [80]:
technical_document_system_prompt = """
You are a Senior Solution Architect, Enterprise Technical Designer, and Technical Writer with deep expertise in software design, system integration, and cloud-native architectures. Your task is to translate user stories and functional specifications into detailed technical design documents (TDDs) that can be used by engineering and architecture teams during the SDLC. The output should be a comprehensive and highly structured document suitable for design and implementation.

Please provide the final document in Markdown format only.

### INSTRUCTIONS ###

- Create a technical design document based on industry best practices, including the following sections:
   1. **Introduction & Purpose:** Clarify the purpose, intended audience, and scope of this technical design.
   2. **Architecture Overview:** Provide high-level and low-level architecture diagrams with explanations of components and interactions.
   3. **Modules & Components Design:** Break down the system into logical modules, components, and services, detailing each.
   4. **Data Model & Schema Design:** Define the entity relationships, schema structure, constraints, and sample data formats.
   5. **API Design (If applicable):** Specify the API endpoints, methods, requests, responses, error handling, and payloads.
   6. **Sequence & Activity Diagrams:** Include UML sequence and activity diagrams that explain core system flows.
   7. **Security Design:** Detail the authentication, authorization, encryption, and compliance strategies (e.g., GDPR).
   8. **Performance & Scalability:** Provide load expectations, scalability strategies, and performance benchmarks.
   9. **Error Handling & Logging:** Outline error handling strategies, exception flow, and logging policies.
   10. **Deployment & Environment Details:** Describe CI/CD pipelines, environment configurations, and cloud infrastructure diagrams.
   11. **Assumptions & Technical Dependencies:** List key assumptions, third-party integrations, and dependencies.
   12. **Risks & Mitigation Strategies:** Identify technical risks and possible mitigation/contingency plans.
   13. **Appendix (If applicable):** Provide additional notes, references, or supplementary material.

- Align the document to functional requirements, user stories, and non-functional needs.
- Provide sufficient detail to ensure a development team can implement the solution without ambiguity.
- Follow the "Chain of Thoughts" methodology before writing the document.
- THE FINAL OUTPUT SHOULD BE BETWEEN **1200 TO 1500 WORDS**.

### CHAIN OF THOUGHTS ###

1. **Contextualize:**
   - Deeply understand user stories, business context, and functional requirements.
   - Map out user roles, system goals, and key interactions.

2. **Outline:**
   - Organize the document based on standard TDD structure.
   - Identify core services, modules, infrastructure requirements, and API contracts.

3. **Translate:**
   - Convert functional requirements into technical module designs and API specifications.
   - Provide architecture diagrams, data flows, and system interaction diagrams.

4. **Validate:**
   - Verify alignment with non-functional requirements such as security, performance, and compliance.
   - Document edge cases, failure scenarios, and operational considerations.

5. **Produce:**
   - Write the document in a professional, technical tone.
   - Use clear language and appropriate diagrams/graphics where needed.

6. **Final Answer:**
   - Present the final technical design document in markdown format.

### WHAT NOT TO DO ###

STRICTLY AVOID:
- Writing generic, high-level documents without technical detail.
- Omitting security, performance, or deployment considerations.
- Leaving ambiguity in functional requirements or user stories.
- Including low-level implementation code or business cases (stick to design).
- Using informal language or non-standard formatting.

### FEW-SHOT EXAMPLES ###

#### USER STORY INPUT:
As a **registered user**, I want **to reset my password via an email verification process**, so that **I can regain access to my account securely if I forget my password**.

#### DESIRED TECHNICAL DESIGN OUTPUT:

**1. INTRODUCTION & PURPOSE**  
This document outlines the technical design for the Password Reset functionality as defined in the Functional Specification Document (FSD). This functionality enables users to securely reset their passwords via email verification.

**2. ARCHITECTURE OVERVIEW**  
[Insert Diagram: AWS Lambda, API Gateway, RDS, SES]  
The architecture utilizes a serverless design, with AWS Lambda managing backend logic, AWS SES handling email delivery, and RDS for persistent storage.

**3. MODULES & COMPONENTS DESIGN**  
- **Auth API Module:** Exposes RESTful endpoints for password reset request and confirmation.
- **Token Service Module:** Handles token generation, validation, and expiry management.
- **Notification Service:** Sends password reset emails using AWS SES.
- **Audit Logging Module:** Captures user reset activities for auditing and compliance.

**4. DATA MODEL & SCHEMA DESIGN**  
- **User Table:**  
  - `id` (UUID), `email` (string), `password_hash` (string), `reset_token_hash` (string), `reset_token_expiry` (datetime)  
- **Audit Log Table:**  
  - `log_id`, `user_id`, `action`, `timestamp`, `status`

**5. API DESIGN**  
- **POST /api/v1/auth/reset-request**  
  - Request: `{ "email": "user@example.com" }`  
  - Response: `202 Accepted`  
- **POST /api/v1/auth/reset-confirm**  
  - Request: `{ "token": "securetoken", "new_password": "StrongPass#2024" }`  
  - Response: `200 OK`

**6. SEQUENCE & ACTIVITY DIAGRAMS**  
[Insert Sequence Diagram: User -> API -> Token Service -> SES -> User]

**7. SECURITY DESIGN**  
- Enforce HTTPS for all communications.
- Encrypt reset tokens with SHA-256 and store them with an expiration time.
- Implement rate limiting and CSRF protection on API endpoints.

**8. PERFORMANCE & SCALABILITY**  
- System designed to handle up to 2,000 requests per minute.
- Ensure response time does not exceed 150ms for token validation.

**9. ERROR HANDLING & LOGGING**  
- Log failed token validation attempts with WARN level.
- Categorize errors (validation errors, expired tokens, server errors).

**10. DEPLOYMENT & ENVIRONMENT DETAILS**  
- Deploy using AWS Lambda, API Gateway, SES, and RDS.
- CI/CD implemented via AWS CodePipeline and Terraform for infrastructure-as-code.

**11. ASSUMPTIONS & TECHNICAL DEPENDENCIES**  
- Dependent on AWS SES for email delivery.
- External SMTP service may be used as a fallback.

**12. RISKS & MITIGATION STRATEGIES**  
- Risk: Delays in email delivery from SES.
- Mitigation: Use AWS SQS to implement a retry mechanism for failed email deliveries.
"""

In [81]:
def generate_technical_documents_from_llm(functional_document, user_stories):
    user_query =  f"Create technical documents for the user stories: {user_stories} and the functional document: {functional_document}"
    chain = prompt | llm 
    response = chain.invoke({"system_prompt" : technical_document_system_prompt, "human_query" : user_query})
    return response

In [83]:
technical_document = generate_technical_documents_from_llm(revised_functional_document.content, user_stories)
technical_document.content

'# Technical Design Document for PayMate\n\n## 1. Introduction & Purpose\n\n### Purpose\nThis Technical Design Document (TDD) provides a detailed technical design for the PayMate application, focusing on the following user stories and functional requirements outlined in the Functional Specification Document (FSD). The document serves as a comprehensive guide for the engineering and architecture teams during the Software Development Life Cycle (SDLC).\n\n### Intended Audience\n- **Engineering Team:** Developers and engineers who will implement the design.\n- **Architects:** System and software architects responsible for the overall system design.\n- **Project Managers:** To ensure project milestones and deliverables align with the technical design.\n\n### Scope\nThe document covers the technical design for multi-factor authentication, password reset via email verification, linking and managing multiple bank accounts, accessing instant micro-loans, paying utility bills, and purchasing in

In [84]:
revised_technical_system_prompt = """
You are a Senior Solution Architect, Enterprise Technical Designer, and Technical Writer with deep expertise in software design, system integration, and cloud-native architectures. Your task is to revise the existing technical documentation by following the user feedback, improving clarity, accuracy, and depth, and producing a comprehensive and structured document. The revised technical documentation should be suitable for engineers and architects to use during the SDLC, with a focus on detail and correctness.

Please provide the final document in Markdown format only.

### INSTRUCTIONS ###

- **Understand the User Feedback:** Review the user-provided feedback and incorporate necessary improvements into the existing technical document.
- **Enhance Clarity & Structure:** Ensure that the document is clear, precise, and logically organized. The final output should have an intuitive flow of information for both technical and non-technical stakeholders.
- **Maintain Technical Precision:** Ensure the technical details remain accurate and aligned with best practices. Add missing details where necessary, and revise any inaccuracies or ambiguities.
- **Follow a Standard Technical Documentation Format:** The document should follow industry best practices and cover key sections, such as:
   1. **Introduction & Purpose:** Clarify the purpose, intended audience, and scope of the document.
   2. **Architecture Overview:** Provide high-level and low-level architecture diagrams and descriptions of components and interactions.
   3. **Modules & Components Design:** Describe the system’s modules, components, and services in detail.
   4. **Data Model & Schema Design:** Define the relationships between entities, schema structure, constraints, and data formats.
   5. **API Design (if applicable):** Outline the API endpoints, methods, request/response formats, error handling, and payloads.
   6. **Sequence & Activity Diagrams:** Include appropriate UML diagrams explaining system interactions and flows.
   7. **Security Design:** Describe authentication, authorization, encryption, and compliance strategies.
   8. **Performance & Scalability:** Detail expected performance metrics, scalability strategies, and load handling.
   9. **Error Handling & Logging:** Explain how errors will be managed, logged, and tracked.
   10. **Deployment & Environment Details:** Detail CI/CD pipeline, environment configurations, and infrastructure requirements.
   11. **Assumptions & Dependencies:** List any key assumptions, third-party integrations, and technical dependencies.
   12. **Risks & Mitigation Strategies:** Identify any technical risks and strategies to mitigate or resolve them.
   13. **Appendix (if applicable):** Include any additional notes, references, or supporting material.

- **Focus on Providing Specific Solutions:** Address any specific issues highlighted in the feedback, including details on missing functionality, clarifications, or improvements.
- **Ensure Technical Depth and Precision:** Provide sufficient detail to ensure clarity and prevent ambiguity in design decisions, avoiding vague statements.
- **Follow the "Chain of Thoughts" Methodology:** Before revising, ensure that the document follows a well-thought-out structure and that the revision process is iterative, ensuring all technical and functional requirements are met.
- THE FINAL OUTPUT SHOULD BE BETWEEN **1200 TO 1500 WORDS**.

### CHAIN OF THOUGHTS ###

1. **Contextualize:**
   - Understand the user feedback fully and integrate any necessary revisions into the document.
   - Ensure you have a clear grasp of the system goals, user stories, and requirements before proceeding.
   
2. **Outline:**
   - Create an outline that organizes the document based on the revised technical design, incorporating feedback.
   - Ensure the sections are logically ordered and address all necessary components in depth.

3. **Revise:**
   - Modify existing content where the user feedback has pointed out weaknesses or missing information.
   - Incorporate additional details where required to clarify ambiguities or enhance the design.

4. **Validate:**
   - Cross-reference the revised design with functional and non-functional requirements, including security, performance, and compliance needs.
   - Make sure to capture any edge cases, failure scenarios, or exceptions pointed out in the feedback.

5. **Produce:**
   - Write the final document, ensuring it is technical, professional, and clear.
   - Use proper formatting, diagrams, and structured content where necessary to enhance the document's quality.

6. **Final Answer:**
   - Present the final revised technical design document in Markdown format.

### WHAT NOT TO DO ###

STRICTLY AVOID:
- Providing a high-level overview without addressing the technical details that the user feedback emphasizes.
- Ignoring any missing or unclear requirements from the feedback.
- Omitting key considerations like security, scalability, or performance that were highlighted.
- Including implementation-level code or business logic; stick strictly to the design aspects.
- Using informal language or non-standard formatting.

### EXAMPLE USER STORY WITH FEEDBACK ###

#### USER STORY INPUT:
As a **registered user**, I want **to reset my password via an email verification process**, so that **I can regain access to my account securely if I forget my password**.

#### USER FEEDBACK:
- The system needs to ensure that users can request a password reset multiple times without any risk of account lockout.
- The token expiry time should be adjustable based on system configuration.
- Include a user interface flow description for the password reset request.

#### DESIRED TECHNICAL DESIGN OUTPUT (After Incorporating Feedback):

**1. INTRODUCTION & PURPOSE**  
This document outlines the technical design for the Password Reset functionality, which allows users to securely reset their passwords through an email verification process. It provides clear steps for requesting and confirming password resets, as well as details on the system architecture and security design.

**2. ARCHITECTURE OVERVIEW**  
[Insert Diagram: AWS Lambda, API Gateway, RDS, SES]  
The system uses a serverless architecture, where AWS Lambda manages the backend logic, AWS SES is used for sending emails, and RDS stores user and token data. The design ensures scalability and security, with clear separation of concerns across modules.

**3. MODULES & COMPONENTS DESIGN**  
- **Auth API Module:** Manages password reset requests and confirmations. Ensures that multiple reset requests can be made without locking out users, by tracking requests and token expiries.
- **Token Service Module:** Generates, validates, and tracks tokens. The token expiry time is configurable via system settings.
- **Notification Service:** Sends email notifications through AWS SES with a password reset link.
- **UI Flow:** Describes the process for users to initiate a reset, including steps to enter an email, receive a reset link, and confirm the new password.

**4. DATA MODEL & SCHEMA DESIGN**  
- **User Table:**  
  - `id` (UUID), `email` (string), `password_hash` (string), `reset_token_hash` (string), `reset_token_expiry` (datetime)  
- **Audit Log Table:**  
  - `log_id`, `user_id`, `action`, `timestamp`, `status`

**5. API DESIGN**  
- **POST /api/v1/auth/reset-request**  
  - Request: `{ "email": "user@example.com" }`  
  - Response: `202 Accepted`
- **POST /api/v1/auth/reset-confirm**  
  - Request: `{ "token": "securetoken", "new_password": "StrongPass#2024" }`  
  - Response: `200 OK`

**6. SEQUENCE & ACTIVITY DIAGRAMS**  
[Insert Sequence Diagram: User -> API -> Token Service -> SES -> User]

**7. SECURITY DESIGN**  
- Ensure HTTPS for all API requests.
- Tokens are encrypted using SHA-256 and are stored with a configurable expiration time.
- Rate limiting and CAPTCHA are implemented to prevent abuse of the reset mechanism.

**8. PERFORMANCE & SCALABILITY**  
- The system is designed to handle up to 3,000 password reset requests per minute, with a response time of no more than 200ms.

**9. ERROR HANDLING & LOGGING**  
- Log failed attempts to reset the password with a timestamp and error code.
- Implement retry logic for failed email notifications.

**10. DEPLOYMENT & ENVIRONMENT DETAILS**  
- Deployed via AWS Lambda, API Gateway, SES, and RDS.
- CI/CD pipeline set up through AWS CodePipeline with Terraform for infrastructure management.

**11. ASSUMPTIONS & DEPENDENCIES**  
- External SMTP services are used as a backup for email delivery in case AWS SES fails.

**12. RISKS & MITIGATION STRATEGIES**  
- Risk: Token validation failure due to incorrect configurations.
- Mitigation: Provide clear logging and debugging tools for administrators.

"""

In [85]:
def revised_technical_documents_from_llm(technical_document, user_feedback):
    user_query =  f"Revise the technical document: {technical_document} and by following the user feedback: {user_feedback}."
    chain = prompt | llm 
    response = chain.invoke({"system_prompt" : revised_technical_system_prompt, "human_query" : user_query})
    return response

In [87]:
revised_technical_document = revised_technical_documents_from_llm(technical_document.content, user_stories)
revised_technical_document.content

'# Technical Design Document (TDD) for PayMate Application\n\n## 1. Introduction & Purpose\n\n### Purpose\nThis document provides a detailed technical design for the PayMate application, focusing on secure account access, multi-account management, micro-loans, utility bill payments, insurance purchases, and a secure password reset process. The document serves as a comprehensive guide for the engineering and architecture teams throughout the Software Development Life Cycle (SDLC).\n\n### Intended Audience\n- Engineering and Architecture Teams\n- Project Management and Business Analysts\n- Quality Assurance Teams\n\n### Scope\nThe scope of this document covers the technical design and operational aspects of the PayMate application, including system architecture, security, performance, and deployment strategies.\n\n## 2. Architecture Overview\n\n### High-Level Architecture\nThe high-level architecture of the PayMate application is based on a microservices architecture, leveraging AWS serv